# LLM API – End-to-End Examples (Single Notebook)

This notebook shows a minimal client and step-by-step examples for:

1. Create a new account
2. Login
3. Change models (admin privilege example)
4. Start a new chat and get a response
5. Continue a chat
6. See chat history
7. Perform a websearch
8. List tools and run math problems
9. Answer based on JSON data ()

Set your API base URL below if different from the default.


In [ ]:
API_BASE_URL = "http://127.0.0.1:8000"
print("Using:", API_BASE_URL)


In [ ]:
import httpx

class LLMApiClient:
    def __init__(self, base_url: str):
        self.base_url = base_url.rstrip("/")
        self.token = None

    def _headers(self):
        h = {"Content-Type": "application/json"}
        if self.token:
            h["Authorization"] = f"Bearer {self.token}"
        return h

    def signup(self, username: str, password: str, role: str = "guest"):
        r = httpx.post(f"{self.base_url}/api/auth/signup", json={
            "username": username, "password": password, "role": role
        })
        r.raise_for_status()
        return r.json()

    def login(self, username: str, password: str):
        r = httpx.post(f"{self.base_url}/api/auth/login", json={
            "username": username, "password": password
        })
        r.raise_for_status()
        data = r.json()
        self.token = data["access_token"]
        return data

    def list_models(self):
        r = httpx.get(f"{self.base_url}/v1/models", headers=self._headers())
        r.raise_for_status()
        return r.json()

    def change_model(self, model: str):
        r = httpx.post(f"{self.base_url}/api/admin/model", json={"model": model}, headers=self._headers())
        r.raise_for_status()
        return r.json()

    def chat_new(self, model: str, user_message: str, agent_type: str = "auto"):
        payload = {
            "model": model,
            "messages": [{"role": "user", "content": user_message}],
            "agent_type": agent_type
        }
        r = httpx.post(f"{self.base_url}/v1/chat/completions", json=payload, headers=self._headers())
        r.raise_for_status()
        data = r.json()
        return data["choices"][0]["message"]["content"], data["x_session_id"]

    def chat_continue(self, model: str, session_id: str, user_message: str, agent_type: str = "auto"):
        payload = {
            "model": model,
            "messages": [{"role": "user", "content": user_message}],
            "session_id": session_id,
            "agent_type": agent_type
        }
        r = httpx.post(f"{self.base_url}/v1/chat/completions", json=payload, headers=self._headers())
        r.raise_for_status()
        data = r.json()
        return data["choices"][0]["message"]["content"], data["x_session_id"]

    def chat_sessions(self):
        r = httpx.get(f"{self.base_url}/api/chat/sessions", headers=self._headers())
        r.raise_for_status()
        return r.json()["sessions"]

    def chat_history(self, session_id: str):
        r = httpx.get(f"{self.base_url}/api/chat/history/{session_id}", headers=self._headers())
        r.raise_for_status()
        return r.json()["messages"]

    def tools(self):
        r = httpx.get(f"{self.base_url}/api/tools/list", headers=self._headers())
        r.raise_for_status()
        return r.json()["tools"]

    def math(self, expression: str):
        r = httpx.post(f"{self.base_url}/api/tools/math", json={"expression": expression}, headers=self._headers())
        r.raise_for_status()
        return r.json()["result"]

    def websearch(self, query: str, max_results: int = 5):
        r = httpx.post(f"{self.base_url}/api/tools/websearch", json={"query": query, "max_results": max_results}, headers=self._headers())
        r.raise_for_status()
        return r.json()["results"]

    def answer_from_json(self, model: str, json_blob: dict, question: str):
        prompt = f"Given this JSON: {json_blob}\nAnswer: {question}"
        return self.chat_new(model, prompt)[0]

client = LLMApiClient(API_BASE_URL)
print("Client ready")


In [ ]:
# 1) Create a new account
username = "alice"
password = "alice_pw"
client.signup(username, password)


In [ ]:
# 2) Login
login = client.login(username, password)
login


In [ ]:
# 3) Change models (admin only) – optional
# client.login("admin", "administrator")
# client.change_model("llama3:8b")


In [ ]:
# List models (OpenAI-compatible)
models = client.list_models()
models


In [ ]:
# 4) Start a new chat and get a response
MODEL = models["data"][0]["id"]
reply, session_id = client.chat_new(MODEL, "Hello! Give me a short haiku about autumn.")
reply, session_id


In [ ]:
# 5) Continue a chat
reply2, _ = client.chat_continue(MODEL, session_id, "Now do one about winter.")
reply2


In [ ]:
# 6) See chat history
client.chat_sessions(), client.chat_history(session_id)


In [ ]:
# 7) Websearch – cite sources in your UI using returned URLs
client.websearch("Who is the president of S. Korea as of October 2025?", max_results=3)


In [ ]:
# 8) Tools: list and math examples
client.tools(), client.math("11.951/3.751"), client.math("max(1.951, 19.51)"), client.math("10 - 2 - 2 + 5 - 1")


In [ ]:
# 9) Answer based on JSON
sample_json = {"users": [{"name": "Tom", "age": 30}, {"name": "Jane", "age": 28}]}
client.answer_from_json(MODEL, sample_json, "Who is older?")


## RAG: Index two Excel files (폴드 긍정.xlsx, 폴드 부정.xlsx)

Place the files under `data/uploads/<your-username>/` or adjust the paths below. The backend now supports `.xlsx` via UnstructuredExcelLoader.


In [ ]:
# Upload (optional): use API /api/files/upload via requests with your JWT if needed.
# Here we assume files are saved server-side and we just trigger indexing by calling the upload API or reading directly if exposed.

import os
from pathlib import Path

# Example local paths on server side (adjust if needed)
username = username  # from earlier cell
pos_path = Path(f"data/uploads/{username}/폴드 긍정.xlsx")
neg_path = Path(f"data/uploads/{username}/폴드 부정.xlsx")

print(pos_path.exists(), neg_path.exists())


In [ ]:
# If files exist, upload via API (to ensure user-scoped indexing)
headers = {"Authorization": f"Bearer {client.token}"}

def upload_file(path: Path):
    files = {"file": (path.name, open(path, "rb"), "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet")}
    r = httpx.post(f"{API_BASE_URL}/api/files/upload", headers=headers, files=files)
    r.raise_for_status()
    return r.json()

uploads = []
if pos_path.exists():
    uploads.append(upload_file(pos_path))
if neg_path.exists():
    uploads.append(upload_file(neg_path))

uploads


In [ ]:
# Query RAG across all indexed docs
r = httpx.get(f"{API_BASE_URL}/api/tools/rag/search", params={"query": "폴드에 대한 긍정/부정 기준을 요약해줘", "top_k": 5}, headers=headers)
r.raise_for_status()
r.json()
